In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/My\ Drive/BDA(모델링)/ws_안심구역데이터


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/BDA(모델링)/ws_안심구역데이터


In [ ]:
import pandas as pd
df= pd.read_csv('./tbsh_gyeonggi_day_202404_용인시.csv')

In [ ]:
df.head()

,ta_ymd,cty_rgn_no,admi_cty_no,card_tpbuz_cd,card_tpbuz_nm_1,card_tpbuz_nm_2,hour,sex,age,day,amt,cnt
0,20240401,41461,41461250,D01,소매/유통,가전제품,1,F,2,1,41604,17
1,20240401,41461,41461250,D01,소매/유통,가전제품,1,F,3,1,971391,189
2,20240401,41461,41461250,D01,소매/유통,가전제품,1,F,4,1,755990,145
3,20240401,41461,41461250,D01,소매/유통,가전제품,1,F,5,1,268970,69
4,20240401,41461,41461250,D01,소매/유통,가전제품,1,F,6,1,260681,67


In [ ]:
df_=df.copy()
df_.columns = ['기준년월일', '시군구코드', '행정동코드', '카드사_업종분류코드', '카드사_업종대분류명', '카드사_업종중분류명','시간대', '성별', '연령별', '요일', '매출금액', '매출건수']
df_.head()

,기준년월일,시군구코드,행정동코드,카드사_업종분류코드,카드사_업종대분류명,카드사_업종중분류명,시간대,성별,연령별,요일,매출금액,매출건수
0,20240401,41461,41461250,D01,소매/유통,가전제품,1,F,2,1,41604,17
1,20240401,41461,41461250,D01,소매/유통,가전제품,1,F,3,1,971391,189
2,20240401,41461,41461250,D01,소매/유통,가전제품,1,F,4,1,755990,145
3,20240401,41461,41461250,D01,소매/유통,가전제품,1,F,5,1,268970,69
4,20240401,41461,41461250,D01,소매/유통,가전제품,1,F,6,1,260681,67


In [ ]:
df_.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2141207 entries, 0 to 2141206
Data columns (total 12 columns):
 #   Column      Dtype 
---  ------      ----- 
 0   기준년월일       int64 
 1   시군구코드       int64 
 2   행정동코드       int64 
 3   카드사_업종분류코드  object
 4   카드사_업종대분류명  object
 5   카드사_업종중분류명  object
 6   시간대         int64 
 7   성별          object
 8   연령별         int64 
 9   요일          int64 
 10  매출금액        int64 
 11  매출건수        int64 
dtypes: int64(8), object(4)
memory usage: 196.0+ MB


In [ ]:
# 나이 70세_이상
df= df[df['age'] >= 7]

## 3-1. 전반적인 소비 확인
- 총 순위
1. 소매/유통
2. 음식
3. 의료/건강
4. 생활서비스

In [ ]:
df_.groupby('카드사_업종대분류명').agg({'매출건수':'sum'}).sort_values(by='매출건수', ascending=False)

,매출건수
카드사_업종대분류명,
소매/유통,13763108
음식,7030610
생활서비스,3025357
의료/건강,2028153
여가/오락,1439971
미디어/통신,364859
공공/기업/단체,310187
학문/교육,255593
공연/전시,13628


In [ ]:
df_[df_['카드사_업종대분류명'] == '소매/유통'].groupby('카드사_업종중분류명').agg({'매출건수':'sum'}).sort_values(by='매출건수',ascending=False)

,매출건수
카드사_업종중분류명,
종합소매점,9071650
음/식료품소매,1210328
가전제품,1119625
제조/도매,578878
의복/의류,543545
화장품소매,355035
사무/교육용품,171245
인테리어/가정용품,150586
선물/완구,149102


In [ ]:
df_[df_['카드사_업종대분류명'] == '의료/건강'].groupby('카드사_업종중분류명').agg({'매출건수':'sum'}).sort_values(by='매출건수',ascending=False)

,매출건수
카드사_업종중분류명,
의약/의료품,916791
일반병원,822320
특화병원,173251
수의업,73607
종합병원,26345
기타의료,15839


## 3-2. 행정동별 소비 확인

In [ ]:
admi_yong_card = df_.groupby(['행정동코드','카드사_업종대분류명']).agg({'매출건수':'sum'}).pivot_table(columns='카드사_업종대분류명',index='행정동코드',values='매출건수')
admi_yong_card

카드사_업종대분류명,공공/기업/단체,공연/전시,미디어/통신,생활서비스,소매/유통,여가/오락,음식,의료/건강,학문/교육
행정동코드,,,,,,,,,
41461250,173030.0,3602.0,86.0,110327.0,1503466.0,839043.0,252080.0,63563.0,6721.0
41461253,28069.0,146.0,22921.0,86817.0,409680.0,47079.0,174300.0,21823.0,2886.0
41461256,3721.0,469.0,202.0,73225.0,338906.0,29599.0,114714.0,14934.0,1492.0
41461259,396.0,99.0,4159.0,71381.0,337138.0,21358.0,129418.0,27906.0,3490.0
41461340,2319.0,429.0,519.0,17258.0,105403.0,708.0,32526.0,5074.0,667.0
41461350,5923.0,13.0,NaN,25176.0,130072.0,3861.0,43782.0,9071.0,349.0
41461360,10352.0,31.0,NaN,151939.0,309149.0,16095.0,151383.0,12696.0,4369.0
41461510,2317.0,NaN,25106.0,124961.0,525094.0,35951.0,296045.0,198110.0,2553.0
41461525,8452.0,166.0,9505.0,60650.0,402682.0,35360.0,284226.0,70084.0,10698.0


In [ ]:
hos_mart_card = admi_yong_card.iloc[:,[4,7]].sort_values(by=['의료/건강','소매/유통'],ascending=False)
hos_mart_card

카드사_업종대분류명,소매/유통,의료/건강
행정동코드,,
41461510,525094.0,198110.0
41465590,630705.0,143332.0
41465520,340508.0,138737.0
41463577,166646.0,116303.0
41463520,557183.0,109146.0
41463590,440418.0,108509.0
41463575,308692.0,94464.0
41465560,523539.0,84993.0
41463510,346574.0,84678.0
